# FastICA Example Application

This notebook is used to test out the the FastICA implementation in the ica.py file, based on the article "Independent component analysis: algorithms and applications" (Hyvarinen and Oja, 2000).

In [ ]:
from ica import FastICA
import numpy as np
import matplotlib.pyplot as plt
from numpy.matlib import repmat
from scipy.signal import sawtooth
from tqdm.notebook import tqdm

In [ ]:
# Dummy example set-up
n_samples = int(1e3)
S = np.array([np.random.rand(n_samples), np.random.rand(n_samples)])
x = np.linspace(0, n_samples-1, n_samples)
S = np.array([np.sin(x), sawtooth(x * 1.9)])
W = np.array([[1, 2], [2, 1]]) # mixing matrix
A = np.linalg.inv(W)
X = A @ S # getting our linear mixtures
n_i = 2 # two independent components
model = FastICA(n_i, X)
model.optimize()

In [ ]:
# Computing the estimation error
W_est = model.get_unwhitened_unmixer()
print("True unmixing matrix: \n", W)
print("\n Estimated unmixing matrix: \n", W_est)

# Plotting Results]
S_est = model.separate_sources(X)
s_mean_est = repmat((W_est @ model.avg.reshape(-1, 1)), 1, X.shape[1])
S = np.divide(S - s_mean_est, S.std(axis=1, keepdims=True))
x = np.linspace(start=0, stop=X.shape[1] - 1, num=n_samples)

fig = plt.figure()
plt.plot(x[:100], S[0, :100], color='red')
plt.title('First Independent Component')
plt.show()

fig2 = plt.figure()
plt.plot(x[:100], S_est[0, :100])
plt.title('First Independent Component (Est.)')
plt.show()

fig3 = plt.figure()
plt.plot(x[:100], S[1, :100], color='red')
plt.title('Second Independent Component')
plt.show()

fig4 = plt.figure()
plt.plot(x[:100], S_est[1, :100])
plt.title('Second Independent Component (Est.)')
plt.show()

# Visualizing the effect of Gaussianity on ICA estimates
Theoretically, as ICA aims to maximise Gaussianity, the more Gaussian the mixtures are, the worse the performance of the estimator will be. Thus, given the CLT, the estimation error should go up and we include more and more uniform r.v.'s in our mixtures. 

The code below tries to split mixtures of uniform distributions, and computes the expected estimation error in each case.

In [ ]:
# Cell containing useful functions to be used
def MSE(x1, x2):
    """ Computes MSE between two numpy arrays."""
    return np.mean(np.square(x1 - x2))

def estimation_error(S, S_est):
    """ Computes estimation error based on the real sources and estimated ICs.
    This function computes the error between each real source and every estimate and
    only includes the lowest error, as ICA does not find ICs in a given order.
    Args:
        S (ndarray): Matrix containing all observations of the real input sources
        S_est (ndarray): Matrix containing all observations of the estimated input sources
    Returns:
        float: estimation error (MSE) between real sources and estimated sources. 
    """
    errors = [] # the MSE for each estimated IC
    for i in range(S.shape[0]): # for each ICA
        these_errors = [MSE(S[i, :], S_est[j, :]) for j in range(S_est.shape[0])]
        errors.append(min(these_errors)) # only keep minimum
        S_est = np.delete(S_est, np.argmin(these_errors), axis=0)
    return np.mean(errors)

In [ ]:
# Setting up (n - 1) different cases with n sources
n_sources = 35
n_samples = int(1e3)
S_total = [np.random.rand(n_samples) for i in range(n_sources)] # contains all possible sources
W_total = [np.random.rand(i,i)*9 + 1 for i in range(2, n_sources + 1)] # contains all unmixing matrices
# Obtain linear mixtures and optimize models
X_total = []
errors = [] # error for ICAs with different number of ICs
for i in tqdm(range(2, n_sources + 1)):
    # Setting up data
    S = np.array(S_total)[:i, :]
    W = W_total[i - 2]
    A = np.linalg.inv(W)
    X = A @ S
    X_total.append(X)
    # Optimizing
    model = FastICA(i, X)
    model.optimize()
    S_est = model.separate_sources(X)
    err = estimation_error(np.divide(S - S.mean(axis=1, keepdims=True), S.std(axis=1, keepdims=True)), S_est)
    errors.append(err)

In [ ]:
# Plotting results
fig = plt.figure()
plt.plot(range(2, n_sources + 1), errors)
plt.xlabel('Number of ICs')
plt.ylabel('Estimation Error')
plt.title('Estimation Error vs Gaussianity of sources')
plt.show()